In [2]:
from matplotlib.patches import Polygon
import numpy as np

Use data: test/pcd_test/hasu_tanashi
Use ROI:

![](static/004_pcd_roi_clip.png)

In [3]:
# need pip install -e . to get into develop mode
import easyidp as idp

In [4]:
# test pcd data
pts = idp.PointCloud(r"../tests/data/pcd_test/hasu_tanashi_ascii.ply")
pts

             x        y        z  r    g    b        nx      ny      nz
    0  -18.908  -15.778   -0.779  123  103  79   nodata  nodata  nodata
    1  -18.908  -15.777   -0.78   124  104  81   nodata  nodata  nodata
    2  -18.907  -15.775   -0.802  123  103  80   nodata  nodata  nodata
  ...  ...      ...      ...      ...  ...  ...     ...     ...     ...
42451  -15.789  -17.961   -0.847  116  98   80   nodata  nodata  nodata
42452  -15.789  -17.939   -0.84   113  95   76   nodata  nodata  nodata
42453  -15.786  -17.937   -0.833  115  97   78   nodata  nodata  nodata

In [13]:
# read roi
test_data = np.loadtxt(r'../tests/data/roi_test/hasu_tanashi_xyz.txt', delimiter=',')

poly_data = np.append(test_data, test_data[0,:][None,:], axis = 0)
poly_data

array([[-18.42576599, -16.10819054,  -0.63814539],
       [-18.00066757, -18.05295944,  -0.67380333],
       [-16.05021095, -17.63488388,  -0.68102068],
       [-16.46848488, -15.66774559,  -0.6401825 ],
       [-18.42576599, -16.10819054,  -0.63814539]])

The above has been wrapped to `roi.read_cc_txt()`

In [17]:
# clip by min-max
xmin, ymin = poly_data[:, 0:2].min(axis=0)
xmax, ymax = poly_data[:, 0:2].max(axis=0)


In [18]:
xy = pts.points[:,0:2]
xy

array([[-18.90831184, -15.77755833],
       [-18.90827942, -15.77727413],
       [-18.90719604, -15.77482891],
       ...,
       [-15.78929043, -17.96125984],
       [-15.78858089, -17.93910408],
       [-15.78621864, -17.93657875]])

In [30]:
xy.shape

(42454, 2)

In [22]:
xy[:,0] >= xmin

array([False, False, False, ...,  True,  True,  True])

In [24]:
inbbox = (xy[:,0] >= xmin) * (xy[:,0] <= xmax) * (xy[:,1] >= ymin) * (xy[:,1]<= ymax)
inbbox

array([False, False, False, ..., False, False, False])

In [31]:
inbbox_idx = np.where(inbbox)[0]
inbbox_idx

array([ 3394,  3395,  3396, ..., 41371, 41372, 41373], dtype=int64)

In [32]:
inbbox_pts = xy[inbbox_idx, :]
inbbox_pts.shape

(31781, 2)

```python
>>> from matplotlib.patches import Polygon
>>> import numpy as np
>>> a = np.array([[2,6],[4,6],[4,8],[2,8],[2,6]])
>>> import matplotlib.pyplot as plt
>>> point = (3,7)
>>> poly = Polygon(a)
>>> poly.contains_point(point)
True
>>> poly.contains_points(np.asarray([[3,7],[12,34]]))
array([ True, False])
```

In [34]:
# judge by matplotlib.polygon
plt_poly = Polygon(poly_data[:,0:2])
in_poly_idx = plt_poly.contains_points(inbbox_pts)
in_poly_idx

array([ True, False, False, ..., False, False, False])

In [35]:
final_pick_idx = inbbox_idx[in_poly_idx]
final_pick_idx

array([ 3394,  3503,  3614, ..., 41265, 41285, 41369], dtype=int64)

In [36]:
len(final_pick_idx)

22423